# 2. Importing Libraries
___
___

In [ ]:
import cv2
import face_recognition
import sqlite3
import numpy as np
from datetime import datetime
import time
import pickle

# 2. Creating Database to try that code is valid
___
___

In [ ]:
def connect_db():
    return sqlite3.connect('session.db', timeout=10)

def create_or_update_table():
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute('''
        CREATE TABLE IF NOT EXISTS attendance (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            NAME TEXT,
            entry_time TEXT,
            exit_time TEXT,
            encoding BLOB
        )
        ''')

        conn.commit()
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

 # 3.  This code logs a person's entry and exit times along with their face encoding in a database.
___
___

In [ ]:
def When_entry(name, encoding):
    try:
        if isinstance(encoding, np.ndarray):
            print(f"Encoding to be stored for {name}: {encoding}")  # طباعة الـ encoding قبل التخزين
            encoding_data = pickle.dumps(encoding)  # ترميز الـ encoding باستخدام pickle
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            conn = connect_db()
            cursor = conn.cursor()
            cursor.execute("INSERT INTO attendance (NAME, entry_time, encoding) VALUES (?, ?, ?)", 
                           (name, current_time, sqlite3.Binary(encoding_data)))
            conn.commit()


            cursor.execute("SELECT encoding FROM attendance WHERE NAME = ?", (name,))
            stored_data = cursor.fetchone()

            if stored_data and stored_data[0] is not None:  
                try:
                    print(f"Stored encoding for {name}: {pickle.loads(stored_data[0])}")  
                except Exception as e:
                    print(f"Error decoding stored encoding for {name}: {e}")
            else:
                print(f"No encoding found or stored encoding is None for {name}")
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

def when_exit(name):
    try:
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("UPDATE attendance SET exit_time = ? WHERE NAME = ? AND exit_time IS NULL", 
                       (current_time, name))
        conn.commit()
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

### 4.The code finds and returns the name of a person whose face matches a given face encoding from the database.
___
___

In [ ]:
def find_matching_face(face_encoding):
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("SELECT NAME, encoding FROM attendance")
        all_faces = cursor.fetchall()

        for stored_name, stored_encoding_blob in all_faces:
            if stored_encoding_blob: 
                try:
                    stored_encoding = pickle.loads(stored_encoding_blob) 

                    matches = face_recognition.compare_faces([stored_encoding], face_encoding)

                    if matches[0]: 
                        return stored_name
                except Exception as e:
                    print(f"Error decoding face encoding for {stored_name}: {e}")
    except sqlite3.OperationalError as e:
        print(f"Database is locked or another operational error occurred: {e}")
    finally:
        conn.close()

    return None


# 4. Loading Haarcascade Model
___

In [ ]:
face_detection = cv2.CascadeClassifier(r'c:\Users\skynet\Pictures\Ml Projects\haarcascade_frontalface_default.xml')

# 5. start the camera
___
___

In [ ]:

camera = cv2.VideoCapture(0)

# 6.Track the presence status (in/out)
___

In [ ]:
person_present = {}
last_seen_time = {}
cooldown_time = 20

# 7.Loop for Face Detection and Attendance Logging
___

In [2]:

create_or_update_table()

while True:
    st, frame = camera.read()
    if not st:
        print("Failed to capture image")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray_frame, 1.3, 5)
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y, x+w, y+h, x)])

        for face_encoding in face_encodings:
            name = find_matching_face(face_encoding)

            if name:
                cv2.putText(frame, f"{name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

                if name not in person_present:
                    When_entry(name, face_encoding)
                    person_present[name] = True

                last_seen_time[name] = time.time()
            else:
                actual_name = input("Enter the name for the new person: ")
                When_entry(actual_name, face_encoding)
                person_present[actual_name] = True
                last_seen_time[actual_name] = time.time()
                cv2.putText(frame, f"{actual_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    for name in list(person_present.keys()):
        if time.time() - last_seen_time.get(name, 0) > cooldown_time:
            when_exit(name)
            del person_present[name]

    cv2.imshow('Camera', frame)

    if cv2.waitKey(33) & 0xff == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


Encoding to be stored for shahd: [-0.08002024  0.09479529  0.0681569  -0.06637253 -0.03428217 -0.00902759
 -0.03725578 -0.13216746  0.19444416 -0.18779844  0.18869168  0.024106
 -0.16486718  0.06274083 -0.05953491  0.14963475 -0.12024979 -0.1352302
 -0.08346893 -0.02789873 -0.0164275   0.04432323 -0.01261143  0.04204717
 -0.15639256 -0.28452903 -0.11808975 -0.0539486   0.07329038 -0.01535869
 -0.06171651  0.01646279 -0.1395956   0.03812061  0.0905349   0.07032199
 -0.01646456 -0.05518173  0.2470955   0.06942674 -0.21236864 -0.06100057
  0.05942722  0.24396278  0.19972004  0.00135879  0.06437322 -0.14295848
  0.10555193 -0.26957902  0.07591765  0.13344507  0.03686685  0.14215675
  0.05395338 -0.14513342  0.05260829  0.15007514 -0.16762738  0.05966691
  0.07618096 -0.05419493 -0.03140914  0.03061246  0.23469386  0.14485206
 -0.11735696 -0.17163315  0.21660523 -0.23020014 -0.03963112  0.07195301
 -0.06299677 -0.17439201 -0.19294779 -0.02522257  0.46178758  0.23673175
 -0.15207525  0.06740